# Day 01 – Core Feature Union 구축

## 목표
- Model A( median 기반 )와 Model B( mean 기반 )에서 선정한 Top-40 core feature를 불러온다.
- 두 리스트의 합집합을 만들어 `feature_union_set`으로 정의한다.
- 전체 데이터에서 `feature_union_set`만 추출한 전용 데이터셋을 만든 뒤, 파일로 저장한다.
- (선택) 이후 mRMR / Boruta를 적용할 준비까지 해둔다.

In [2]:
from pathlib import Path
import pandas as pd
import json

# 랜덤 시드 (필요하면 사용)
RANDOM_STATE = 42

# 프로젝트 루트 기준 경로 설정 (필요하면 수정해서 사용)
ROOT_DIR = Path("..")  # 노트북 기준 위치에 맞게 조정
RESULTS_DIR = ROOT_DIR / "results"
STAGEC_DIR = RESULTS_DIR / "stageC"

# core feature 리스트 파일 경로 (이미 존재하는 파일)
MODEL_A_CORE_PATH = STAGEC_DIR / "core_features_median_top40.json"  # Model A (median)
MODEL_B_CORE_PATH = STAGEC_DIR / "core_features_mean_top40.json"    # Model B (mean)

# 전체(전처리 완료) 데이터 경로  👉 여기 본인 파일명에 맞게 꼭 수정해줘!
DATA_DIR = ROOT_DIR / "data"
FULL_DATA_PATH = DATA_DIR / "processed" / "base_master_mean.parquet"  # TODO: 실제 파일명으로 변경

# feature_union_set 저장 경로
STAGEF_DIR = RESULTS_DIR / "stageF"
STAGEF_DIR.mkdir(parents=True, exist_ok=True)


print("ROOT_DIR   :", ROOT_DIR.resolve())
print("RESULTS_DIR:", RESULTS_DIR.resolve())
print("STAGEC_DIR :", STAGEC_DIR.resolve())
print("존재 여부  :", STAGEC_DIR.exists())

print(MODEL_A_CORE_PATH)
print(MODEL_B_CORE_PATH)
print(FULL_DATA_PATH)
print(STAGEF_DIR)

ROOT_DIR   : /Users/mealkuo/Desktop/capstone02_project
RESULTS_DIR: /Users/mealkuo/Desktop/capstone02_project/results
STAGEC_DIR : /Users/mealkuo/Desktop/capstone02_project/results/stageC
존재 여부  : True
../results/stageC/core_features_median_top40.json
../results/stageC/core_features_mean_top40.json
../data/processed/base_master_mean.parquet
../results/stageF


## 1. Model A / Model B core feature 리스트 불러오기

- `core_features_median_top40.json` : Model A (median + Robust)
- `core_features_mean_top40.json`   : Model B (mean + Standard)
- 두 리스트를 Python list로 로딩한다.

In [3]:
# JSON 파일에서 core feature 리스트 읽어오기
with open(MODEL_A_CORE_PATH, "r") as f:
    core_features_median_top40 = json.load(f)["features"]   # ← 핵심: features만 꺼냄

with open(MODEL_B_CORE_PATH, "r") as f:
    core_features_mean_top40 = json.load(f)["features"]     # ← 핵심: features만 꺼냄

# 출력 확인
print("Model A (median) core features 갯수:", len(core_features_median_top40))
print("Model B (mean)   core features 갯수:", len(core_features_mean_top40))

print("\n[Model A sample]", core_features_median_top40[:5])
print("[Model B sample]", core_features_mean_top40[:5])

Model A (median) core features 갯수: 39
Model B (mean)   core features 갯수: 28

[Model A sample] ['sensor_065', 'sensor_430', 'sensor_206', 'sensor_342', 'sensor_104']
[Model B sample] ['sensor_130', 'sensor_060', 'sensor_368', 'sensor_175', 'sensor_103']


## 2. Model A + B core feature 합집합 생성 (`feature_union_set`)

- 두 리스트의 union을 구해 **feature_union_set**으로 정의한다.
- 이후 모든 고급 실험(Stage F~)의 기본 후보 피처 셋이 된다.

In [ ]:
# 합집합(Union) 생성
feature_union_set = sorted(list(set(core_features_median_top40) | set(core_features_mean_top40)))

print("feature_union_set 총 갯수:", len(feature_union_set))
print("feature_union_set 샘플:", feature_union_set[:10])

feature_union_set 총 갯수: 62
feature_union_set 샘플: ['sensor_011', 'sensor_015', 'sensor_041', 'sensor_042', 'sensor_052', 'sensor_060', 'sensor_064', 'sensor_065', 'sensor_068', 'sensor_072']


## 3. 전체 데이터에서 feature_union_set 컬럼만 추출

- 전처리 완료된 전체 데이터셋(`FULL_DATA_PATH`)을 로드한다.
- `feature_union_set`에 포함된 컬럼만 선택해서 **feature_union_set용 DataFrame**을 만든다.
- 라벨 컬럼(`label`)은 그대로 유지한다. (필요시 `target_col` 이름 수정)

In [5]:
# TODO: 라벨 컬럼 이름이 다르면 여기 수정해줘
target_col = "label"

# 전체 데이터 로드 (전처리 끝난 버전: mean 또는 median)
full_df = pd.read_parquet(FULL_DATA_PATH)
print("full_df shape:", full_df.shape)

# feature_union_set에 라벨이 섞여 있으면 제거
if target_col in feature_union_set:
    feature_union_set.remove(target_col)

# 🔥 union 중 full_df에 존재하는 컬럼만 선택 (핵심)
feature_union_set_filtered = [c for c in feature_union_set if c in full_df.columns]

# 전체 union 중 제거된 컬럼 출력
removed_cols = list(set(feature_union_set) - set(feature_union_set_filtered))
if len(removed_cols) > 0:
    print("전처리 중 제거된 컬럼들:", removed_cols)
else:
    print("전처리로 제거된 컬럼 없음")

print(f"필터링된 union feature 수: {len(feature_union_set_filtered)}")

# feature_union_set_filtered + label만 남긴 데이터셋 생성
feature_union_df = full_df[feature_union_set_filtered + [target_col]].copy()

print("feature_union_df shape:", feature_union_df.shape)
feature_union_df.head()

full_df shape: (1567, 251)
전처리 중 제거된 컬럼들: ['sensor_148', 'sensor_198', 'sensor_249', 'sensor_456', 'sensor_206', 'sensor_213', 'sensor_430', 'sensor_406', 'sensor_183', 'sensor_429', 'sensor_492']
필터링된 union feature 수: 51
feature_union_df shape: (1567, 52)


,sensor_011,sensor_015,sensor_041,sensor_042,sensor_052,sensor_060,sensor_064,sensor_065,sensor_068,sensor_072,...,sensor_477,sensor_486,sensor_501,sensor_524,sensor_541,sensor_561,sensor_574,sensor_575,sensor_585,label
0,-0.0034,7.9558,61.29,4.515,218.3174,-1.7264,16.1445,21.7264,0.9226,84.0793,...,31.9893,178.1759,0.0000,0.1094,4.0825,0.1039,0.3157,3.0624,0.0035,0
1,-0.0148,10.1548,78.25,2.773,205.1695,0.8073,10.9036,19.1927,1.1598,82.3494,...,30.8643,359.0444,0.0000,0.0673,2.9799,0.0375,0.2653,2.0111,0.0055,0
2,0.0013,9.5157,14.37,5.434,185.7574,23.8245,11.3019,16.1755,0.8694,84.7681,...,13.3923,190.3869,0.0000,0.0751,2.9667,0.1233,0.1882,4.0923,0.0039,1
3,-0.0033,9.6052,76.90,1.279,189.9079,24.3791,13.5597,15.6209,0.9761,70.2289,...,35.4323,88.5553,711.6418,0.0977,3.2465,0.0592,0.1738,2.8971,0.0025,0
4,-0.0072,10.5661,76.39,2.209,233.5491,-12.2945,21.9782,32.2945,0.9256,65.8417,...,42.6838,240.1781,0.0000,0.0616,1.9999,0.0461,0.2224,3.1776,0.0036,0


## 4. feature_union_set 결과 저장

- 리스트(컬럼 이름들): `feature_union_set_columns.json`
- 데이터셋: `feature_union_set.parquet`
- 저장 위치: `results/stageF/`

In [6]:
# 1) feature_union_set 컬럼 이름 저장 (filtered 버전으로 저장!)
UNION_FEATURE_LIST_PATH = STAGEF_DIR / "feature_union_set_columns.json"
with open(UNION_FEATURE_LIST_PATH, "w") as f:
    json.dump(feature_union_set_filtered, f, indent=2)

# 2) feature_union_set 데이터셋 저장
UNION_FEATURE_DF_PATH = STAGEF_DIR / "feature_union_set.parquet"
feature_union_df.to_parquet(UNION_FEATURE_DF_PATH, index=False)

print("✅ feature_union_set 저장 완료")
print("컬럼 리스트  :", UNION_FEATURE_LIST_PATH)
print("데이터셋 파일:", UNION_FEATURE_DF_PATH)

✅ feature_union_set 저장 완료
컬럼 리스트  : ../results/stageF/feature_union_set_columns.json
데이터셋 파일: ../results/stageF/feature_union_set.parquet


## 5. mRMR vs Boruta 적용 준비

## 5. mRMR / Boruta 적용 준비 및 Core Feature 재선정

- Day02에서 사용할 `feature_union_set.parquet`를 로드하고,
- Boruta 알고리즘(랜덤포레스트 기반)을 활용해
  - union feature들 중에서 최종 Core Feature Set(대략 30~50개)을 재선정한다.
- 선정된 최종 코어 피처는 `core_features_final_stageF.json`으로 저장한다.

In [7]:
import pandas as pd
from pathlib import Path
import json

# ==== 경로 설정 (위에서 이미 했으면 생략 가능) ====
ROOT_DIR = Path("..")
RESULTS_DIR = ROOT_DIR / "results"
STAGEF_DIR = RESULTS_DIR / "stageF"

# Day01에서 저장한 feature_union_set 경로
UNION_FEATURE_DF_PATH = STAGEF_DIR / "feature_union_set.parquet"

# label 컬럼 이름 (다르면 수정)
target_col = "label"

# ==== feature_union_set 로드 ====
feature_union_df = pd.read_parquet(UNION_FEATURE_DF_PATH)
print("feature_union_df shape:", feature_union_df.shape)

# X, y 분리
X_union = feature_union_df.drop(columns=[target_col])
y_union = feature_union_df[target_col].values

print(f"Feature 수: {X_union.shape[1]}")
print(f"Sample 수 : {X_union.shape[0]}")
X_union.head()

feature_union_df shape: (1567, 52)
Feature 수: 51
Sample 수 : 1567


,sensor_011,sensor_015,sensor_041,sensor_042,sensor_052,sensor_060,sensor_064,sensor_065,sensor_068,sensor_072,...,sensor_449,sensor_477,sensor_486,sensor_501,sensor_524,sensor_541,sensor_561,sensor_574,sensor_575,sensor_585
0,-0.0034,7.9558,61.29,4.515,218.3174,-1.7264,16.1445,21.7264,0.9226,84.0793,...,0.1362,31.9893,178.1759,0.0000,0.1094,4.0825,0.1039,0.3157,3.0624,0.0035
1,-0.0148,10.1548,78.25,2.773,205.1695,0.8073,10.9036,19.1927,1.1598,82.3494,...,0.2751,30.8643,359.0444,0.0000,0.0673,2.9799,0.0375,0.2653,2.0111,0.0055
2,0.0013,9.5157,14.37,5.434,185.7574,23.8245,11.3019,16.1755,0.8694,84.7681,...,0.2748,13.3923,190.3869,0.0000,0.0751,2.9667,0.1233,0.1882,4.0923,0.0039
3,-0.0033,9.6052,76.90,1.279,189.9079,24.3791,13.5597,15.6209,0.9761,70.2289,...,0.3332,35.4323,88.5553,711.6418,0.0977,3.2465,0.0592,0.1738,2.8971,0.0025
4,-0.0072,10.5661,76.39,2.209,233.5491,-12.2945,21.9782,32.2945,0.9256,65.8417,...,0.7592,42.6838,240.1781,0.0000,0.0616,1.9999,0.0461,0.2224,3.1776,0.0036


### 5.1 Boruta 기반 Feature Selection

- mRMR는 정보 이론 기반의 필터 방식이고,  
  Boruta는 RandomForest를 기반으로 한 wrapper 방식의 feature selection 기법이다.
- 여기서는 구현 및 라이브러리 의존성을 고려해서 **Boruta**를 우선 사용하고,  
  필요 시 mRMR(mutual information 기반 간단 버전)을 추가할 수 있도록 구조를 만든다.

> Boruta를 사용하려면 `boruta` 또는 `boruta_py` 라이브러리가 필요하다.  
> 예시:  
> `pip install boruta`  또는  `pip install boruta_py`

In [8]:
from sklearn.ensemble import RandomForestClassifier

try:
    # boruta 패키지 (pip install boruta)
    from boruta import BorutaPy
    BORUTA_AVAILABLE = True
except ImportError:
    print("⚠️ 'boruta' 라이브러리가 설치되어 있지 않습니다. 먼저 'pip install boruta'를 해주세요.")
    BORUTA_AVAILABLE = False

RANDOM_STATE = 42

if BORUTA_AVAILABLE:
    # ==== Boruta용 RandomForest 설정 ====
    rf = RandomForestClassifier(
        n_estimators=300,
        max_depth=None,
        n_jobs=-1,
        class_weight="balanced_subsample",
        random_state=RANDOM_STATE,
    )

    # BorutaPy 설정
    feat_selector = BorutaPy(
        estimator=rf,
        n_estimators='auto',      # 자동으로 적정 트리 수 탐색
        two_step=True,
        verbose=2,
        random_state=RANDOM_STATE
    )

    print("🔁 Boruta feature selection 시작...")
    feat_selector.fit(X_union.values, y_union)

    # 선택된 피처 마스크
    selected_mask = feat_selector.support_
    tentative_mask = feat_selector.support_weak_

    selected_features = X_union.columns[selected_mask].tolist()
    tentative_features = X_union.columns[tentative_mask].tolist()

    print("\n✅ Boruta 완료")
    print(f" - 확정 선택된 피처 수: {len(selected_features)}")
    print(f" - tentative(애매한) 피처 수: {len(tentative_features)}")

    # 상위 30~50개 선에서 자르고 싶다면(선택)
    # 여기서는 일단 Boruta가 확정 선택한 전체를 core로 두고,
    # 필요 시 상위 N개로 후처리할 수 있도록 한다.
else:
    selected_features = []
    tentative_features = []

🔁 Boruta feature selection 시작...
Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	51
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	51
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	51
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	51
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	51
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	51
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	51
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	5
Tentative: 	13
Rejected: 	33
Iteration: 	9 / 100
Confirmed: 	5
Tentative: 	13
Rejected: 	33
Iteration: 	10 / 100
Confirmed: 	5
Tentative: 	13
Rejected: 	33
Iteration: 	11 / 100
Confirmed: 	5
Tentative: 	13
Rejected: 	33
Iteration: 	12 / 100
Confirmed: 	5
Tentative: 	10
Rejected: 	36
Iteration: 	13 / 100
Confirmed: 	5
Tentative: 	10
Rejected: 	36
Iteration: 	14 / 100
Confirmed: 	5
Tentative: 	10
Rejected: 	36
Iteration: 	15 / 100
Confirmed: 	5
Tentative: 	10
Rejected: 	36
Iteration: 	16 / 100
Co

### 5.2 최종 Core Feature Set 저장

- Boruta로 확정 선정된 피처 리스트를  
  `core_features_final_stageF.json` 으로 저장한다.
- 이후 StageF~G 모델링에서 이 피처들을 기반으로
  LightGBM / RF / Ensemble 모델을 학습한다.

In [9]:
if len(selected_features) == 0:
    print("⚠️ 선택된 피처가 없습니다. Boruta가 정상적으로 실행되었는지 확인하세요.")
else:
    CORE_FEATURE_FINAL_PATH = STAGEF_DIR / "core_features_final_stageF.json"

    core_feature_log = {
        "n_selected_features": len(selected_features),
        "selected_features": selected_features,
        "n_tentative_features": len(tentative_features),
        "tentative_features": tentative_features,
        "boruta_params": {
            "estimator": "RandomForestClassifier",
            "n_estimators": "auto",
            "class_weight": "balanced_subsample",
            "random_state": RANDOM_STATE
        }
    }

    with open(CORE_FEATURE_FINAL_PATH, "w", encoding="utf-8") as f:
        json.dump(core_feature_log, f, indent=2, ensure_ascii=False)

    print("✅ 최종 Core Feature Set 저장 완료")
    print(" - 저장 경로:", CORE_FEATURE_FINAL_PATH)
    print(" - 선택된 feature 수:", len(selected_features))
    print(" - 예시:", selected_features[:10])

✅ 최종 Core Feature Set 저장 완료
 - 저장 경로: ../results/stageF/core_features_final_stageF.json
 - 선택된 feature 수: 6
 - 예시: ['sensor_060', 'sensor_103', 'sensor_104', 'sensor_130', 'sensor_342', 'sensor_426']


### 5.3 mRMR 기반 Feature Selection

- mRMR(minimum Redundancy Maximum Relevance)은  
  **"중복은 최소화하고 타깃과의 연관성은 최대화"**하는 필터 기반 방법이다.
- Mutual Information(MI)를 사용하여 각 Feature의 중요도를 평가하고,
- Redundancy(피처 간 상관성)를 차감하는 방식으로 점수를 조정한다.
- Boruta보다 빠른 대신, 모델 기반 안정성은 상대적으로 낮다.
- Day02에서는 Boruta와 mRMR을 모두 계산해서 Core Feature 후보를 교차 검증할 수 있다.

In [10]:
from sklearn.feature_selection import mutual_info_classif
import numpy as np

def mrmr_selection(X_df, y, top_k=40):
    """
    간단한 mRMR 구현:
    1) mutual information으로 relevance 계산
    2) 피처 간 상관성으로 redundancy penalty 적용
    """
    cols = X_df.columns.tolist()
    
    # --- 1) relevance = MI(feature, label)
    mi = mutual_info_classif(X_df, y, discrete_features=False, random_state=42)
    relevance = dict(zip(cols, mi))
    
    # --- 2) redundancy 계산: feature 간 상관성의 평균
    corr_matrix = X_df.corr().abs()
    
    selected = []
    candidates = set(cols)
    
    while len(selected) < top_k and len(candidates) > 0:
        mrmr_scores = {}
        
        for feat in candidates:
            # redundancy: 이전에 고른 feature들과의 평균 상관도
            if len(selected) == 0:
                redundancy = 0
            else:
                redundancy = corr_matrix.loc[feat, selected].mean()
            
            # mRMR 점수 = relevance - redundancy
            mrmr_scores[feat] = relevance[feat] - redundancy
        
        # 가장 점수가 높은 피처 선택
        best_feat = max(mrmr_scores, key=mrmr_scores.get)
        selected.append(best_feat)
        candidates.remove(best_feat)
    
    return selected, relevance

In [11]:
TOP_K = 40  # 또는 30~50 범위 내에서 선택할 수 있음

print(f"🔍 mRMR 기반 Feature Selection 실행 (Top-{TOP_K})...")

mrmr_selected, mi_scores = mrmr_selection(X_union, y_union, top_k=TOP_K)

print("\n✅ mRMR 완료")
print(f" - 선택된 feature 수: {len(mrmr_selected)}")
print(f" - 예시 10개: {mrmr_selected[:10]}")

🔍 mRMR 기반 Feature Selection 실행 (Top-40)...

✅ mRMR 완료
 - 선택된 feature 수: 40
 - 예시 10개: ['sensor_042', 'sensor_575', 'sensor_367', 'sensor_414', 'sensor_332', 'sensor_561', 'sensor_175', 'sensor_416', 'sensor_501', 'sensor_068']


In [12]:
MRMR_OUTPUT_PATH = STAGEF_DIR / f"core_features_mrmr_top{TOP_K}.json"

mrmr_log = {
    "top_k": TOP_K,
    "n_selected_features": len(mrmr_selected),
    "selected_features": mrmr_selected,
    "mi_scores": mi_scores,  # 원하면 저장
}

with open(MRMR_OUTPUT_PATH, "w", encoding="utf-8") as f:
    json.dump(mrmr_log, f, indent=2, ensure_ascii=False)

print("💾 mRMR 결과 저장 완료:", MRMR_OUTPUT_PATH)

💾 mRMR 결과 저장 완료: ../results/stageF/core_features_mrmr_top40.json


In [13]:
# Boruta 결과: selected_features (확정), tentative_features (애매)
# mRMR 결과: mrmr_selected, mi_scores
# target_col, feature_union_df, STAGEF_DIR 는 이미 위에서 정의되어 있다고 가정

print("Boruta 확정:", len(selected_features))
print("Boruta tentative:", len(tentative_features))
print("mRMR selected:", len(mrmr_selected))

# 1) Boruta 확정 + mRMR 상위 조합
boruta_core = set(selected_features)
mrmr_core = set(mrmr_selected)

# 우선 단순 합집합으로 Core 후보 생성
final_core_candidates = sorted(boruta_core | mrmr_core)
print("초기 Core 후보 수 (Boruta ∪ mRMR):", len(final_core_candidates))

# 2) 실제 데이터셋에 존재하는 컬럼만 필터 (안전장치)
cols_in_df = [c for c in final_core_candidates if c in feature_union_df.columns]
print("feature_union_df에 실제 존재하는 Core 후보 수:", len(cols_in_df))

# 3) 너무 많으면 mutual information 기준 상위 N개로 자르기 (예: 40개)
TOP_K_FINAL = min(40, len(cols_in_df))

# mi_scores는 dict 형태라고 가정 (mRMR 코드에서 반환)
# 없는 컬럼은 MI=0으로 처리
scored = [(col, mi_scores.get(col, 0.0)) for col in cols_in_df]
scored_sorted = sorted(scored, key=lambda x: x[1], reverse=True)

final_core_features = [col for col, score in scored_sorted[:TOP_K_FINAL]]

print("\n✅ 최종 Core Feature 확정")
print(f" - 최종 Core Feature 수: {len(final_core_features)}")
print(f" - 예시 10개: {final_core_features[:10]}")

Boruta 확정: 6
Boruta tentative: 3
mRMR selected: 40
초기 Core 후보 수 (Boruta ∪ mRMR): 43
feature_union_df에 실제 존재하는 Core 후보 수: 43

✅ 최종 Core Feature 확정
 - 최종 Core Feature 수: 40
 - 예시 10개: ['sensor_042', 'sensor_332', 'sensor_407', 'sensor_575', 'sensor_574', 'sensor_041', 'sensor_413', 'sensor_541', 'sensor_175', 'sensor_501']


### StageF Core Dataset 저장 완료
 - 경로: ../results/stageF/stageF_core_dataset.parquet
 - Shape: (1567, 41)

In [14]:
# 최종 Core Feature + label 만 남긴 StageF 데이터셋
stageF_df = feature_union_df[final_core_features + [target_col]].copy()

STAGEF_CORE_PATH = STAGEF_DIR / "stageF_core_dataset.parquet"
stageF_df.to_parquet(STAGEF_CORE_PATH, index=False)

print("✅ StageF Core Dataset 저장 완료")
print(" - 경로:", STAGEF_CORE_PATH)
print(" - Shape:", stageF_df.shape)

# 간단 확인
stageF_df.head()

✅ StageF Core Dataset 저장 완료
 - 경로: ../results/stageF/stageF_core_dataset.parquet
 - Shape: (1567, 41)


,sensor_042,sensor_332,sensor_407,sensor_575,sensor_574,sensor_041,sensor_413,sensor_541,sensor_175,sensor_501,...,sensor_292,sensor_068,sensor_280,sensor_299,sensor_011,sensor_078,sensor_097,sensor_286,sensor_338,label
0,4.515,0.0752,14.2396,3.0624,0.3157,61.29,24.8504,4.0825,0.3184,0.0000,...,0.0167,0.9226,0.0319,0.0373,-0.0034,0.0141,-0.2786,1.5123,1.5989,0
1,2.773,0.0778,10.5837,2.0111,0.2653,78.25,15.0150,2.9799,0.4115,0.0000,...,0.0139,1.1598,0.0440,0.0502,-0.0148,0.0004,0.5854,2.1016,1.1506,0
2,5.434,0.0243,11.4871,4.0923,0.1882,14.37,18.0233,2.9667,0.3141,0.0000,...,0.0167,0.8694,0.0249,0.0800,0.0013,-0.0052,-0.1343,1.5306,1.5481,1
3,1.279,0.0243,13.3972,2.8971,0.1738,76.90,17.8537,3.2465,0.2618,711.6418,...,0.0245,0.9761,0.0417,0.0285,-0.0033,-0.0400,0.0411,2.8960,1.7317,0
4,2.209,0.0822,7.1493,3.1776,0.2224,76.39,22.3369,1.9999,0.3053,0.0000,...,0.0181,0.9256,0.0257,0.0755,-0.0072,-0.0167,0.2189,3.2376,1.6216,0


### 최종 core feature 기준으로 smote + LightGBM 성능 확인해보기

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score, average_precision_score
from imblearn.over_sampling import SMOTE
from lightgbm import LGBMClassifier
import pandas as pd

# StageF Core Dataset 로드
STAGEF_CORE_PATH = STAGEF_DIR / "stageF_core_dataset.parquet"
df = pd.read_parquet(STAGEF_CORE_PATH)

target_col = "label"
X = df.drop(columns=[target_col])
y = df[target_col].values

print("데이터 shape:", df.shape)

# 1) Train/Test 8:2 split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 2) 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 3) SMOTE 적용 (기본)
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train_scaled, y_train)

print("Resampled Train shape:", X_train_res.shape)
print("Positive ratio:", y_train_res.mean())

# 4) LightGBM baseline
model = LGBMClassifier(
    n_estimators=600,
    learning_rate=0.05,
    max_depth=-1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)

model.fit(X_train_res, y_train_res)

# 5) 평가
y_pred = model.predict(X_test_scaled)
y_prob = model.predict_proba(X_test_scaled)[:, 1]

print("\n====== Stage F Test 성능 ======")
print(classification_report(y_test, y_pred, digits=4))
print("ROC-AUC :", roc_auc_score(y_test, y_prob))
print("PR-AUC  :", average_precision_score(y_test, y_prob))

데이터 shape: (1567, 41)
Resampled Train shape: (2340, 40)
Positive ratio: 0.5
[LightGBM] [Info] Number of positive: 1170, number of negative: 1170
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000609 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 10189
[LightGBM] [Info] Number of data points in the train set: 2340, number of used features: 40
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further split

/Users/mealkuo/Desktop/capstone02_project/capstone02/lib/python3.13/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/mealkuo/Desktop/capstone02_project/capstone02/lib/python3.13/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [16]:
import pandas as pd
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    balanced_accuracy_score, confusion_matrix
)
from sklearn.metrics import roc_auc_score, average_precision_score

def summarize_metrics(y_true, y_pred, y_prob):
    # 혼동행렬
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    
    # 기본 지표
    acc  = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, zero_division=0)
    rec  = recall_score(y_true, y_pred, zero_division=0)
    f1   = f1_score(y_true, y_pred, zero_division=0)
    
    # 추가 지표
    spec      = tn / (tn + fp) if (tn + fp) > 0 else 0  # Specificity
    bal_acc   = balanced_accuracy_score(y_true, y_pred)
    roc_auc   = roc_auc_score(y_true, y_prob)
    pr_auc    = average_precision_score(y_true, y_prob)
    
    # 표로 정리
    df_metrics = pd.DataFrame({
        "Metric": [
            "Accuracy",
            "Precision",
            "Recall",
            "Specificity",
            "F1 Score",
            "Balanced Accuracy",
            "ROC-AUC",
            "PR-AUC",
            "TN", "FP", "FN", "TP"
        ],
        "Value": [
            acc,
            prec,
            rec,
            spec,
            f1,
            bal_acc,
            roc_auc,
            pr_auc,
            tn, fp, fn, tp
        ]
    })
    return df_metrics

metric_table = summarize_metrics(y_test, y_pred, y_prob)
display(metric_table)

,Metric,Value
0,Accuracy,0.910828
1,Precision,0.111111
2,Recall,0.047619
3,Specificity,0.972696
4,F1 Score,0.066667
5,Balanced Accuracy,0.510158
6,ROC-AUC,0.724525
7,PR-AUC,0.148576
8,TN,285.000000
9,FP,8.000000


In [18]:
import json
from pathlib import Path

# ------------------------------------------------------
# 1) 최종 피처 리스트 추출
# ------------------------------------------------------
feature_list = list(X_train.columns)
print(f"✔ 총 {len(feature_list)}개의 feature 추출 완료.")

# ------------------------------------------------------
# 2) models 폴더 경로 자동 설정
#    (StageH 노트북이 보통 capstone02_project/notebooks 에 있으므로)
# ------------------------------------------------------
project_root = Path.cwd().parents[0]   # notebooks 폴더 기준 → project root
models_dir = project_root / "models"
models_dir.mkdir(exist_ok=True)

output_path = models_dir / "stageI_final_features.json"

# ------------------------------------------------------
# 3) JSON 저장
# ------------------------------------------------------
with open(output_path, "w", encoding="utf-8") as f:
    json.dump({"feature_list": feature_list}, f, ensure_ascii=False, indent=2)

print(f"🎉 feature_list.json 저장 완료 → {output_path}")

✔ 총 40개의 feature 추출 완료.
🎉 feature_list.json 저장 완료 → /Users/mealkuo/Desktop/capstone02_project/models/stageI_final_features.json
